<a href="https://colab.research.google.com/github/caioitalo/soulcode/blob/main/Mkt_campaign_Atividade_Pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXTRAÇÃO DE DADOS

In [ ]:
#baixando a biblioteca pandera
!pip install pandera



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#importando as bibliotecas
import pandas as pd
import pandera as pa

In [ ]:
# lendo o DF a partir de um Dataset hospedado em um bucket na GCP
df = pd.read_csv('https://storage.googleapis.com/datalake-soulcode-ed7/arquivos%20soulcode/marketing_campaign.csv%20-%20marketing_campaign.csv.csv', sep=',')

In [ ]:
#criando um backup antes de tratar 
df2 = df.copy()

In [ ]:
df

,ano_nasc,escolaridade,estado_civil,renda_anual,filhos_crianca,filhos_adolesc,cliente_desde,ultima_compra,compra_vinhos,compra_frutas,...,compra_peixes,compra_doces,compra_ouro_prod,compras_acordo,compras_web,compras_catalogo,compras_loja,site_visita_mes,reclamacoes,tot_filhos
id,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduado,Solteiro,58138.0,0,0,2012-09-04,58,635,88,...,172,88,88,3,8,10,4,7,0,0
2174,1954,Graduado,Solteiro,46344.0,1,1,2014-03-08,38,11,1,...,2,1,6,2,1,1,2,5,0,2
4141,1965,Graduado,Amasiado,71613.0,0,0,2013-08-21,26,426,49,...,111,21,42,1,8,2,10,4,0,0
6182,1984,Graduado,Amasiado,26646.0,1,0,2014-02-10,26,11,4,...,10,3,5,2,2,0,4,6,0,1
5324,1981,PhD,Casado,58293.0,1,0,2014-01-19,94,173,43,...,46,27,15,5,5,3,6,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10870,1967,Graduado,Casado,61223.0,0,1,2013-06-13,46,709,43,...,42,118,247,2,9,3,4,5,0,1
4001,1946,PhD,Amasiado,64014.0,2,1,2014-06-10,56,406,0,...,0,0,8,7,8,2,5,7,0,3
7270,1981,Graduado,Divorciado,56981.0,0,0,2014-01-25,91,908,48,...,32,12,24,1,2,3,13,6,0,0


In [ ]:
df.dtypes

ano_nasc                    object
escolaridade                object
estado_civil                object
renda_anual                float64
filhos_crianca               int64
filhos_adolesc               int64
cliente_desde       datetime64[ns]
ultima_compra                int64
compra_vinhos                int64
compra_frutas                int64
compra_carnes                int64
compra_peixes                int64
compra_doces                 int64
compra_ouro_prod             int64
compras_acordo               int64
compras_web                  int64
compras_catalogo             int64
compras_loja                 int64
site_visita_mes              int64
reclamacoes                  int64
tot_filhos                   int64
dtype: object

#TRANSFORMAÇÃO DOS DADOS

In [ ]:
# renomeando as colunas
df.rename(columns={'ID':'id','Year_Birth':'ano_nasc','Education':'escolaridade','Marital_Status':'estado_civil','Income':'renda_anual',
                   'Kidhome':'filhos_crianca','Teenhome':'filhos_adolesc','Dt_Customer':'cliente_desde','Recency':'ultima_compra','MntWines':'compra_vinhos',
                   'MntFruits':'compra_frutas','MntMeatProducts':'compra_carnes','MntFishProducts':'compra_peixes','MntSweetProducts':'compra_doces',
                   'MntGoldProds':'compra_ouro_prod','NumDealsPurchases':'compras_acordo','NumWebPurchases':'compras_web',
                   'NumCatalogPurchases':'compras_catalogo','NumStorePurchases':'compras_loja','AcceptedCmp1':'cmp1_aceito','AcceptedCmp2':'cmp2_aceito',
                   'AcceptedCmp3':'cmp3_aceito','AcceptedCmp4':'cmp4_aceito','AcceptedCmp5':'cmp5_aceito','Complain':'reclamacoes',
                   'Z_CostContact':'custo_contrato','Z_Revenue':'receita','Response':'resposta','NumWebVisitsMonth':'site_visita_mes'},inplace=True)

In [ ]:
#checando dados vazios
df.isna().sum()

id                   0
ano_nasc             0
escolaridade         0
estado_civil         0
renda_anual         24
filhos_crianca       0
filhos_adolesc       0
cliente_desde        0
ultima_compra        0
compra_vinhos        0
compra_frutas        0
compra_carnes        0
compra_peixes        0
compra_doces         0
compra_ouro_prod     0
compras_acordo       0
compras_web          0
compras_catalogo     0
compras_loja         0
site_visita_mes      0
cmp3_aceito          0
cmp4_aceito          0
cmp5_aceito          0
cmp1_aceito          0
cmp2_aceito          0
reclamacoes          0
custo_contrato       0
receita              0
resposta             0
dtype: int64

In [ ]:
# setando coluna id como rótulo e verificando se ela é única
df.set_index(['id'], verify_integrity=True, inplace=True)

In [ ]:
#transformando a coluna cliente_desde para o tipo data
df['cliente_desde'] = pd.to_datetime(df['cliente_desde'],dayfirst=True)

In [ ]:
# dropando custo_contrato e receita que possuem os mesmos valores, portanto não contribuem para nenhuma análise
# as demais colunas dropei pois as análises desejadas não serão afetadas por elas
df.drop(['cmp1_aceito','cmp2_aceito','cmp3_aceito','cmp4_aceito','cmp5_aceito','resposta'], axis=1,inplace=True)
df.drop(['custo_contrato', 'receita'],axis=1,inplace=True)


In [ ]:
# checando inconsistências
# inconsistências encontradas: nasc (1893, 1899, 1900), estado_civil: alone, absurd, YOLO; renda 666666
# receita e custo do contrato têm sempre os mesmos valores -> drop
df.groupby(['estado_civil'],dropna=False).size().sort_values(ascending=False)

estado_civil
Casado        864
Amasiado      580
Solteiro      483
Divorciado    232
Viúvo(a)       77
NaN             4
dtype: int64

In [ ]:
# traduzindo os elementos das colunas
# o replace das datas são inconsistentes pois é improvável pessoas com 120+ comprando na web. provável digitação errada/outliers
# YOLO e absurd não tem como definir, também teve que ser removido

df['escolaridade'] = df['escolaridade'].replace({'Graduation':'Graduado','Master':'Mestre','2n Cycle':'Especialista','Basic':'Básico'})
df['estado_civil'] = df['estado_civil'].replace({'Married':'Casado', 'Together':'Amasiado', 'Single':'Solteiro','Divorced':'Divorciado',
                                                 'Widow':'Viúvo(a)', 'Alone':'Solteiro'})
df['ano_nasc'].replace([1893,1899,1900],pd.NA,inplace=True)
df['estado_civil'].replace(['Absurd','YOLO'],pd.NA,inplace=True)

In [ ]:
# por alguma razão ao colocar NA no ano_nasc o transformou em object, transformando novamente em int. Inclusive uma dúvida.
df['ano_nasc'] = df['ano_nasc'].astype('Int64')

In [ ]:
# fazendo uma coluna com o total de filhos
df['tot_filhos'] = df['filhos_crianca'] + df['filhos_adolesc']

In [ ]:
df.dtypes

ano_nasc                     Int64
escolaridade                object
estado_civil                object
renda_anual                float64
filhos_crianca               int64
filhos_adolesc               int64
cliente_desde       datetime64[ns]
ultima_compra                int64
compra_vinhos                int64
compra_frutas                int64
compra_carnes                int64
compra_peixes                int64
compra_doces                 int64
compra_ouro_prod             int64
compras_acordo               int64
compras_web                  int64
compras_catalogo             int64
compras_loja                 int64
site_visita_mes              int64
reclamacoes                  int64
tot_filhos                   int64
dtype: object

In [ ]:
#criando um esquema de validação de dados
schema = pa.DataFrameSchema({
    'ano_nasc':pa.Column(int, nullable=True),
    'escolaridade': pa.Column(str),
    'estado_civil': pa.Column(str, nullable=True),
    'renda_anual': pa.Column(float, nullable=True),
    'filhos_crianca': pa.Column(int),
    'filhos_adolesc': pa.Column(int),
    'cliente_desde': pa.Column(pa.DateTime),
    'ultima_compra': pa.Column(int),
    'compra_vinhos': pa.Column(int),
    'compra_frutas':pa.Column(int),
    'compra_carnes':pa.Column(int),
    'compra_peixes':pa.Column(int),
    'compra_doces':pa.Column(int),
    'compra_ouro_prod':pa.Column(int),
    'compras_acordo':pa.Column(int),
    'compras_web':pa.Column(int),
    'compras_catalogo':pa.Column(int),
    'compras_loja':pa.Column(int),
    'site_visita_mes':pa.Column(int),
    'reclamacoes':pa.Column(int),
    'tot_filhos':pa.Column(int)
    }
)



In [ ]:
# checando a validação dos dados
schema.validate(df)

,ano_nasc,escolaridade,estado_civil,renda_anual,filhos_crianca,filhos_adolesc,cliente_desde,ultima_compra,compra_vinhos,compra_frutas,...,compra_peixes,compra_doces,compra_ouro_prod,compras_acordo,compras_web,compras_catalogo,compras_loja,site_visita_mes,reclamacoes,tot_filhos
id,,,,,,,,,,,,,,,,,,,,,
5524,1957,Graduado,Solteiro,58138.0,0,0,2012-09-04,58,635,88,...,172,88,88,3,8,10,4,7,0,0
2174,1954,Graduado,Solteiro,46344.0,1,1,2014-03-08,38,11,1,...,2,1,6,2,1,1,2,5,0,2
4141,1965,Graduado,Amasiado,71613.0,0,0,2013-08-21,26,426,49,...,111,21,42,1,8,2,10,4,0,0
6182,1984,Graduado,Amasiado,26646.0,1,0,2014-02-10,26,11,4,...,10,3,5,2,2,0,4,6,0,1
5324,1981,PhD,Casado,58293.0,1,0,2014-01-19,94,173,43,...,46,27,15,5,5,3,6,5,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10870,1967,Graduado,Casado,61223.0,0,1,2013-06-13,46,709,43,...,42,118,247,2,9,3,4,5,0,1
4001,1946,PhD,Amasiado,64014.0,2,1,2014-06-10,56,406,0,...,0,0,8,7,8,2,5,7,0,3
7270,1981,Graduado,Divorciado,56981.0,0,0,2014-01-25,91,908,48,...,32,12,24,1,2,3,13,6,0,0


In [ ]:
df.dtypes

ano_nasc                     Int64
escolaridade                object
estado_civil                object
renda_anual                float64
filhos_crianca               int64
filhos_adolesc               int64
tot_filhos                   int64
cliente_desde       datetime64[ns]
ultima_compra                int64
site_visita_mes              int64
compra_vinhos                int64
compra_frutas                int64
compra_carnes                int64
compra_peixes                int64
compra_doces                 int64
compra_ouro_prod             int64
compras_acordo               int64
compras_web                  int64
compras_catalogo             int64
compras_loja                 int64
reclamacoes                  int64
dtype: object

In [ ]:
# trocando as colunas de lugar para deixar algo mais racional
df = df[['ano_nasc','escolaridade','estado_civil','renda_anual','filhos_crianca','filhos_adolesc','tot_filhos','cliente_desde','ultima_compra',
        'site_visita_mes','compra_vinhos','compra_frutas','compra_carnes','compra_peixes','compra_doces','compra_ouro_prod','compras_acordo',
        'compras_web','compras_catalogo','compras_loja','reclamacoes']]

# CARREGAMENTO DOS DADOS

In [ ]:
# carregando pra um novo arquivo
df.to_csv('bd_mkt.csv')

# ANÁLISES BÁSICAS

In [ ]:
df.dtypes

ano_nasc                     Int64
escolaridade                object
estado_civil                object
renda_anual                float64
filhos_crianca               int64
filhos_adolesc               int64
tot_filhos                   int64
cliente_desde       datetime64[ns]
ultima_compra                int64
site_visita_mes              int64
compra_vinhos                int64
compra_frutas                int64
compra_carnes                int64
compra_peixes                int64
compra_doces                 int64
compra_ouro_prod             int64
compras_acordo               int64
compras_web                  int64
compras_catalogo             int64
compras_loja                 int64
reclamacoes                  int64
dtype: object

In [ ]:
filtro = df['escolaridade'].unique()
filtro2 = df['renda_anual']
dfesc = df.loc[filtro & filtro2]